# SGD processing

Jupyter notebook for processing SDG data: https://github.com/do-me/SDG

1. Download SDG targets & indicator table from un.org and load in pandas
2. Process columns, separate numbers from targets & indicators 
3. Add SDG Names 

Repeatable at any time. 

In [1]:
import pandas as pd

excel_file = "https://unstats.un.org/sdgs/indicators/Global%20Indicator%20Framework%20after%202023%20refinement.English.xlsx " 
# or use the file in the repo 'Global Indicator Framework after 2023 refinement.English.xlsx'

# Read the Excel file and skip the first two rows
df = pd.read_excel(excel_file, header=2)

# Clean up
del df["Unnamed: 0"]

df.rename(columns={'Goals and targets (from the 2030 Agenda for Sustainable Development)':'Targets'}, inplace=True)
df['Targets'].fillna(method='ffill', inplace=True) # fill the combined excel cells with last valid entry

# if 11.c still doesn#t hold any UNSD code, mask it with 0 
if df.loc[df["Targets"].str.contains("11.c"),"UNSD Indicator Codes†"].isnull().item():
    df.loc[df["Targets"].str.contains("11.c"),"UNSD Indicator Codes†"] = 0

df = df[~df["UNSD Indicator Codes†"].isnull()]

c:\Users\dome\Anaconda3\envs\py3.11\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
# split numbers from text
df["TargetsNumber"] = df["Targets"].apply(lambda x: x.split(" ")[0])
df["Targets"] = df.apply(lambda x: x.Targets.replace(x.TargetsNumber,""), axis=1)
df["IndicatorsNumber"] = df["Indicators"].apply(lambda x: x.split(" ")[0])
df["Indicators"] = df.apply(lambda x: x.Indicators.replace(x.IndicatorsNumber,""), axis=1)
df["SDGNumber"] = df.TargetsNumber.apply(lambda x: x.split(".")[0])

In [3]:
# add SDG names
sdg_names = pd.read_excel("17_SDGs_names.xlsx")
sdg_names["SDGNumber"] = sdg_names["Goal"].apply(lambda x: x.replace("SDG ",""))
del sdg_names["Goal"]
df = df.merge(sdg_names, on="SDGNumber")
df = df[["SDGNumber","SDG Title","TargetsNumber", "Targets", "IndicatorsNumber","Indicators","UNSD Indicator Codes†"]]
df.rename(columns={'SDG Title': 'SDGTitle','UNSD Indicator Codes†': 'UNSDIndicatorCodes'}, inplace=True)

In [4]:
# as of 2023, 11.c has no indicators
if df.loc[df.TargetsNumber == "11.c", "UNSDIndicatorCodes"].item() == 0: # means, there is no indicator and neither UNSD code
    df.loc[df.TargetsNumber == "11.c", "IndicatorsNumber"] = 0
    df.loc[df.TargetsNumber == "11.c", "Indicators"] = ""

In [5]:
# Check only problematic line. IndicatorsNumber	Indicators	UNSDIndicatorCodes should be 0 or ""
df.loc[df.TargetsNumber == "11.c"]

,SDGNumber,SDGTitle,TargetsNumber,Targets,IndicatorsNumber,Indicators,UNSDIndicatorCodes
155,11,Sustainable Cities and Communities,11.c,"Support least developed countries, including ...",0,,0


In [6]:
df.to_excel("SDG_Target_Indicator_UNSD_2023.xlsx", index=False)

In [7]:
df.to_csv("SDG_Target_Indicator_UNSD_2023.csv", index=False, encoding="utf-8", sep=";")

## Load the table

In [8]:
pd.read_csv("SDG_Target_Indicator_UNSD_2023.csv",delimiter=";")

,SDGNumber,SDGTitle,TargetsNumber,Targets,IndicatorsNumber,Indicators,UNSDIndicatorCodes
0,1,No Poverty,1.1,"By 2030, eradicate extreme poverty for all pe...",1.1.1,Proportion of the population living below the...,C010101
1,1,No Poverty,1.2,"By 2030, reduce at least by half the proporti...",1.2.1,Proportion of population living below the nat...,C010201
2,1,No Poverty,1.2,"By 2030, reduce at least by half the proporti...",1.2.2,"Proportion of men, women and children of all ...",C010202
3,1,No Poverty,1.3,Implement nationally appropriate social prote...,1.3.1,Proportion of population covered by social pr...,C010301
4,1,No Poverty,1.4,"By 2030, ensure that all men and women, in pa...",1.4.1,Proportion of population living in households...,C010401
...,...,...,...,...,...,...,...
244,17,Partnerships for the Goals,17.18,"By 2020, enhance capacity-building support to...",17.18.1,Statistical capacity indicator for Sustainabl...,C171804
245,17,Partnerships for the Goals,17.18,"By 2020, enhance capacity-building support to...",17.18.2,Number of countries that have national statis...,C171802
246,17,Partnerships for the Goals,17.18,"By 2020, enhance capacity-building support to...",17.18.3,Number of countries with a national statistic...,C171803
247,17,Partnerships for the Goals,17.19,"By 2030, build on existing initiatives to dev...",17.19.1,Dollar value of all resources made available ...,C171901
